# Earthquake Detections using Computer Vision

This is a little notebook created to demonstrate how the presented algorithm works. The notebook has access to four images which it will use to showcase the algorithm

In [ ]:
import image_processing as ip
import pipeline
import os, pathlib
import matplotlib.pyplot as plt
import numpy as np

## Step-by-step Processing

First, we'll go through the process step by step and see the results.

Let's first take a look at the four images that we have as examples

In [ ]:
# Read images in as RGBA images

image_names = os.listdir("example_images")
image_paths = []
for image in image_names:
    image_paths.append("example_images" / pathlib.Path(image))

images = []
for image in image_paths:
    images.append(ip.read_image_from_file(image, as_gray=False))

fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True, dpi=300)

axes = axes.flatten()
x_range = (50, 1750)
y_range = (130, 1100)

for _i, ax in enumerate(axes):
    ax.set_yticks([])
    ax.set_xticks([])
    ax.set_title(image_names[_i], fontsize=6)
    
    ax.imshow(images[_i], cmap="seismic")
plt.tight_layout()
plt.show()



We can see that we have a variety of different signals here.

Now let's read these images and convert them to grayscale to prepare them for the algorithm. To increase visibility, we will invert the colorscale, the algorithm is working on the normal colorscale though. This should reproduce the figures from the paper.

In [ ]:
images = []
for image in image_paths:
    images.append(ip.read_image_from_file(image, as_gray=True))

fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True, dpi=300)
axes = axes.flatten()
x_range = (50, 1750)
y_range = (130, 1100)

for _i, ax in enumerate(axes):
    images[_i] = ip.crop_image(images[_i], x_range=x_range, y_range=y_range)
    ax.set_yticks([])
    ax.set_xticks([])
    ax.set_title(image_names[_i], fontsize=6)
    img = ax.imshow(images[_i])
    img.set_cmap('gray_r')
plt.tight_layout()
plt.show()

We have now read in the images in grayscale and we can start processing them. The first thing we'll do is to reduce the noise level by removing the median brightness per channel. Remember that the colorscale is inverted so in this case it will look like removing the median darkness per channel.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True, dpi=300)
axes = axes.flatten()

for _i, ax in enumerate(axes):
    images[_i] = ip.remove_median_brightness(images[_i])
    ax.set_yticks([])
    ax.set_xticks([])
    ax.set_title(image_names[_i], fontsize=6)
    img = ax.imshow(images[_i])
    img.set_cmap('gray_r')
plt.tight_layout()
plt.show()


We can now use Otsu's method to define a brightness threshold deciding what is "foreground" and what is "background", we can thus create a binary image.

A good explanation of how Otsu's method works can be found [here](https://www.youtube.com/watch?v=jUUkMaNuHP8&)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True, dpi=300)
axes = axes.flatten()

for _i, ax in enumerate(axes):
    thresholds = ip.compute_brightness_thresholds(images[_i], classes=4)
    images[_i] = ip.apply_brightness_threshold(images[_i], threshold=thresholds[0])
    ax.set_yticks([])
    ax.set_xticks([])
    ax.set_title(image_names[_i], fontsize=6)
    img = ax.imshow(images[_i])
    img.set_cmap('gray_r')
plt.tight_layout()
plt.show()

Ok, now we are actually starting to see some things quite clearly. If we look closely, we can find three earthquakes in these four pictures. The obvious one is in the top right, but there is a small one in the top left too and a very small one in the bottom left of the picture on the bottom right.

We can see how the coherency of the signals from the earthquakes is much higher than the rest of the signals. We can use this information when we apply the coherency threshold in the next step.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True, dpi=300)
axes = axes.flatten()

for _i, ax in enumerate(axes):
    images[_i] = ip.coherency_thresholding(images[_i], min_cluster_size=64)
    ax.set_yticks([])
    ax.set_xticks([])
    ax.set_title(image_names[_i], fontsize=6)
    img = ax.imshow(images[_i])
    img.set_cmap('gray_r')
plt.tight_layout()
plt.show()    

This makes quite a difference as you can see. We have gotten rid of a lot of noisy signal while keeping signal from all three earthquakes in there. You can also notice now that all the signal in there is contained in small horizontal lines. We can thus try creating a template which looks a bit like that and remove those lines from the images.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True, dpi=300)
axes = axes.flatten()

# It can make things more intuitive to relate the temple size with the size of 
# the actual images
cols = images[0].shape[1]
# It needs to be of an integer size so we use floor division
template = ip.create_shaped_template(vertical_length=1, horizontal_length=cols // 200)

for _i, ax in enumerate(axes):
    images[_i] = ip.remove_template_shape(images[_i].astype(np.float32), template=template)
    ax.set_yticks([])
    ax.set_xticks([])
    ax.set_title(image_names[_i], fontsize=6)
    img = ax.imshow(images[_i])
    img.set_cmap('gray_r')
plt.tight_layout()
plt.show()   

You can see that the image is cleaner than before but not fully clean.
Those horizontal lines are much less coherent though, so we can apply the coherency threshold again and see what happens.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, sharex=True, sharey=True, dpi=300)
axes = axes.flatten()

for _i, ax in enumerate(axes):
    images[_i] = ip.coherency_thresholding(images[_i], min_cluster_size=64)
    ax.set_yticks([])
    ax.set_xticks([])
    ax.set_title(image_names[_i], fontsize=6)
    img = ax.imshow(images[_i])
    img.set_cmap('gray_r')
plt.tight_layout()
plt.show()

Now all we have left are the signals from the Earthquakes. We can count the True signals in each image

In [ ]:
reg_props = []
for i in range(len(images)):
    reg_props.append(ip.get_regionprops(images[i]))
    print(f"Image: {image_names[i]} has {len(reg_props[i])} True regions")

Which checks out to what we estimated.

## All at once

We can do all of this without all the plotting, and just get a quick answer from the image. This can be done with either a data file or an image.

We do not provide a data reader as data readers can contain proprietary information, so here we will only demonstrate the pipeline using images.

In [ ]:
for _i, image in enumerate(image_paths):
    quake = pipeline.earthquake_in_image(
        filename = image,
        x_range = (50, 1750),
        y_range = (130, 1100),
        median=True,
        mean=False,
        classes=4,
        threshold=0,
        min_cluster_size=64,
        templates=(1, 1700 // 200)
    )
    if quake:
        print(f"Image {image_names[_i]} includes an earthquake")
    else:
        print(f"Image {image_names[_i]} does not include an earthquake")


Which is the same answer as we got from the step-by-step showcase.